##                                      **PRACTICA BIG DATA II**

---



  Alumno: Neri KOUTCHALA




###                                    CLASIFICACIÓN NON BALANCEADA



**1. INSTALAMOS EL ENTORNO SPARK**

In [2]:
#Primero instalamos Apache Spark con Hadoop
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz 

#Instalamos los paquetes de Python para trabajar con Spark
!pip install findspark #Instalamos FindSpark
!pip install pyspark   #Instalamos Spark

#Indicamos a PySpark donde está Spark
import findspark
findspark.init("spark-3.2.3-bin-hadoop2.7")#SPARK_HOME

#Inicializamos las variables de entorno
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop2.7"



#Creamos una sesión de Spark para poder trabajar
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Clasificación non Balanceada con PySpark") \
    .getOrCreate() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=7dccbfb075bb15a7a9da569ec5afd860782d9ab55e825ad2d0779e110787802c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


**2. CARGA DE LAS BIBLIOTECAS** 

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType
import pandas as pd


**3. DESCARGAMOS, CARGAMOS Y PROCESAMOS LOS DATOS**

In [4]:
#Descargamos los ficheros de datos
!wget -nv --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HOrM49tCLA_NqHyD_ps_cv483FPN7aWo' -O susy-10k-tra.csv
!wget -nv --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HT80d5cwU7HMi2XK8CNgxgHvxRZEZB_d' -O susy-10k-tst.csv

#Leemos los conjuntos de entrenamiento y test
dfTra = spark.read.csv('susy-10k-tra.csv', inferSchema=True, header=True)
dfTst = spark.read.csv('susy-10k-tst.csv', inferSchema=True, header=True)


2023-05-22 08:34:34 URL:https://doc-08-bc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qtfftouitt30nlhoalvqcsoiatmnp8g5/1684744425000/11180625338828972622/*/1HOrM49tCLA_NqHyD_ps_cv483FPN7aWo?e=download&uuid=7b44dd42-7dcb-415e-ad34-a63a42f13e04 [3463157/3463157] -> "susy-10k-tra.csv" [1]
2023-05-22 08:34:35 URL:https://doc-14-bc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/imcr599loob8a4acqrfuktjld25ss7be/1684744425000/11180625338828972622/*/1HT80d5cwU7HMi2XK8CNgxgHvxRZEZB_d?e=download&uuid=1b8bce1e-12ab-4570-9d16-964fa3479094 [3469204/3469204] -> "susy-10k-tst.csv" [1]


In [5]:
#Convertimos la variable clase en entera
trainData=dfTra.withColumn("clase",dfTra.clase.cast("Integer"))
testData=dfTst.withColumn("clase",dfTst.clase.cast("Integer"))

In [6]:
trainData.show(5)
testData.show(5)

+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+-----+
|               uno|                dos|                tres|             cuatro|               cinco|               seis|              siete|                ocho|              nueve|                diez|              once|              doce|             trece|            catorce|            quince|         dieciseis|        diecisiete|          dieciocho|clase|
+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+---------

In [7]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

# Definimos los nombres de las columnas input y output
input_cols = trainData.columns[:-1]
output_col = "atributos"
 
#Unimos las variables de entrada con VectorAssembler
assembler = VectorAssembler(inputCols=trainData.columns[:-1],outputCol="atributos")
train = assembler.transform(trainData)
train.select('atributos','clase').show(5)

#Uso el mismo VectorAssemble para el conjunto de test (las columnas son iguales)
test = assembler.transform(testData)
test.select('atributos','clase').show(5)

+--------------------+-----+
|           atributos|clase|
+--------------------+-----+
|[0.64334195852279...|    1|
|[1.23746168613433...|    1|
|[1.34331297874450...|    0|
|[0.65310519933700...|    0|
|[1.55166482925415...|    1|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|           atributos|clase|
+--------------------+-----+
|[1.67953264713287...|    1|
|[0.85768365859985...|    1|
|[0.67941337823867...|    1|
|[0.76688265800476...|    1|
|[0.48874709010124...|    0|
+--------------------+-----+
only showing top 5 rows



**4. DEFINIMOS EL TRATAMIENTO ROS Y RUS**

In [8]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


# Convertir el DataFrame de Spark en DataFrame de Pandas
train_data_pd = train.toPandas()

# Aplicar el algoritmo ROS para equilibrar los datos de formación
ros = RandomOverSampler()
train_data_balanced, _ = ros.fit_resample(train_data_pd[input_cols], train_data_pd["clase"])
train_data_balanced = pd.concat([train_data_balanced, train_data_pd["clase"]], axis=1)

# Eliminar filas con valores NaN
train_data_balanced = train_data_balanced.dropna()

# Aplicar el algoritmo RUS para equilibrar aún más los datos de entrenamiento
rus = RandomUnderSampler()
train_data_balanced, _ = rus.fit_resample(train_data_balanced[input_cols], train_data_balanced["clase"])
train_data_balanced = pd.concat([train_data_balanced, train_data_pd["clase"]], axis=1)

# Crear un nuevo Spark DataFrame a partir del Pandas DataFrame equilibrado
train_data_balanced = spark.createDataFrame(train_data_balanced)

# Transforma los datos de entrenamiento equilibrados con VectorAssembler
train_data_balanced = assembler.transform(train_data_balanced)

# Mostrar los datos de entrenamiento equilibrados
train_data_balanced.select(output_col, "clase").show(5)


/content/spark-3.2.3-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+-----+
|           atributos|clase|
+--------------------+-----+
|[0.66922366619110...|    1|
|[0.75525563955307...|    1|
|[2.45638990402221...|    0|
|[0.30020308494567...|    0|
|[0.63320702314376...|    1|
+--------------------+-----+
only showing top 5 rows



**5. DEFINIMOS LAS ETAPAS Y LA TUBERÍA**

In [9]:
#Unimos las variables de entrada con VectorAssembler
assembler = VectorAssembler(inputCols=train.columns[:-1],outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

#label_indexer = StringIndexer(inputCol=output_col, outputCol="clase")
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="clase")

#Añadimos el preprocesamiento a una tubería
from pyspark.ml import Pipeline
tuberiaParcial = Pipeline().setStages([assembler, scaler,  lr])

#Construimos el modelo
model = tuberiaParcial.fit(train)

#Transformamos los datos y los mostramos
datosPreprocesados = model.transform(train)
datosPreprocesados.select('features','clase').show(5)

+--------------------+-----+
|            features|clase|
+--------------------+-----+
|[0.64334195852279...|    1|
|[1.23746168613433...|    1|
|[1.34331297874450...|    0|
|[0.65310519933700...|    0|
|[1.55166482925415...|    1|
+--------------------+-----+
only showing top 5 rows



**6. EVALUACIÓN DEL MODELO**

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Aplicar el modelo al conjunto de datos de prueba para generar predicciones
predictions = model.transform(test)

# Evaluar el modelo
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="clase")
area_under_roc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

# Calcular la tasa de verdaderos positivos (TPR)
tp = predictions.filter("prediction = 1.0 AND clase = 1.0").count()
fn = predictions.filter("prediction = 0.0 AND clase = 1.0").count()
tpr = tp / (tp + fn)

# Calcular la tasa de verdaderos negativos (TNR)
tn = predictions.filter("prediction = 0.0 AND clase = 0.0").count()
fp = predictions.filter("prediction = 1.0 AND clase = 0.0").count()
tnr = tn / (tn + fp)

# Calcular el producto TPR x TNR
tpr_tnr_product = tpr * tnr

# Imprimir los resultados de la evaluación
print("Area under ROC:", area_under_roc)
print("True Positive Rate (TPR):", tpr)
print("True Negative Rate (TNR):", tnr)
print("TPR x TNR Product:", tpr_tnr_product)


Area under ROC: 0.9999998208691394
True Positive Rate (TPR): 1.0
True Negative Rate (TNR): 1.0
TPR x TNR Product: 1.0


**7. APLICACIÓN DEL APRENDIZAJE AUTOMÁTICO**

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, LogisticRegression, LinearSVC
from tabulate import tabulate

# Definir los nombres de las columnas de entrada y salida
input_col = "atributos"
output_col = "features"

# Definir los clasificadores
classifiers = [
    DecisionTreeClassifier(featuresCol=output_col, labelCol='clase'),
    RandomForestClassifier(featuresCol=output_col, labelCol='clase'),
    GBTClassifier(featuresCol=output_col, labelCol='clase'),
    LogisticRegression(featuresCol=output_col, labelCol='clase'),
    LinearSVC(featuresCol=output_col, labelCol='clase')
]

# Definir el evaluador
evaluator = MulticlassClassificationEvaluator(labelCol="clase", metricName="accuracy")

# Crear una lista vacía para almacenar los resultados de la evaluación
results = []

# Iterar sobre los clasificadores
for classifier in classifiers:
    # Crear el VectorAssembler para cada clasificador
    assembler = VectorAssembler(inputCols=[input_col], outputCol="vectorized_features")

    # Crear el MinMaxScaler para cada clasificador
    scaler = MinMaxScaler(inputCol="vectorized_features", outputCol=output_col)

    # Crear el pipeline para cada clasificador
    pipeline = Pipeline(stages=[assembler, scaler, classifier])

    # Ajustar el pipeline a los datos de entrenamiento
    model = pipeline.fit(train)

    # Aplicar la canalización a los datos de prueba
    predictions = model.transform(test)

    # Evaluar las métricas
    accuracy = evaluator.evaluate(predictions)
    f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
    precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

    # Almacenar los resultados de la evaluación
    results.append([classifier.__class__.__name__, accuracy, f1_score, precision, recall])

# Preparar las cabeceras de las tablas
headers = ["Classifier", "Accuracy", "F1 Score", "Precision", "Recall"]

# Imprimir los resultados de la evaluación en forma de tabla
print(tabulate(results, headers=headers, tablefmt="grid"))


+------------------------+------------+------------+-------------+----------+
| Classifier             |   Accuracy |   F1 Score |   Precision |   Recall |
+========================+============+============+=============+==========+
| DecisionTreeClassifier |     0.7876 |   0.786893 |    0.786315 |   0.7876 |
+------------------------+------------+------------+-------------+----------+
| RandomForestClassifier |     0.7937 |   0.793975 |    0.794273 |   0.7937 |
+------------------------+------------+------------+-------------+----------+
| GBTClassifier          |     0.7964 |   0.797172 |    0.798137 |   0.7964 |
+------------------------+------------+------------+-------------+----------+
| LogisticRegression     |     0.8081 |   0.807603 |    0.807187 |   0.8081 |
+------------------------+------------+------------+-------------+----------+
| LinearSVC              |     0.8128 |   0.8111   |    0.810316 |   0.8128 |
+------------------------+------------+------------+------------

**8. VALIDACIÓN**

In [35]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, LogisticRegression, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Fusionar los conjuntos de datos de entrenamiento y de prueba en un único marco de datos
df = dfTra.union(dfTst)

# Crear un EvaluadorClasificaciónMulticlase
evaluator = MulticlassClassificationEvaluator(labelCol="clase", metricName="accuracy")

# Definir los clasificadores
classifiers = [
    DecisionTreeClassifier(featuresCol="atributos", labelCol='clase'),
    RandomForestClassifier(featuresCol="atributos", labelCol='clase'),
    GBTClassifier(featuresCol="atributos", labelCol='clase'),
    LogisticRegression(featuresCol="atributos", labelCol='clase'),
    LinearSVC(featuresCol="atributos", labelCol='clase')
]

# Crear una lista vacía para almacenar los resultados de la validación cruzada
results = []

# Realizar una validación cruzada para cada clasificador
for classifier in classifiers:
    # Definir la malla de parámetros para la validación cruzada
    param_grid = ParamGridBuilder().build()

    # Crear el CrossValidator
    crossval = CrossValidator(estimator=classifier,
                              estimatorParamMaps=param_grid,
                              evaluator=evaluator,
                              numFolds=10)

    # Entrenar el modelo
    cvModel = crossval.fit(df)

    # Almacenar la precisión del modelo
    accuracy = cvModel.avgMetrics[0]
    results.append([classifier.__class__.__name__, accuracy])

# Preparar las cabeceras de las tablas
headers = ["Classifier", "Accuracy"]

# Imprimir los resultados de validación cruzada en forma de tabla
print(tabulate(results, headers=headers, tablefmt="grid"))


+------------------------+------------+
| Classifier             |   Accuracy |
+========================+============+
| DecisionTreeClassifier |   0.767222 |
+------------------------+------------+
| RandomForestClassifier |   0.769482 |
+------------------------+------------+
| GBTClassifier          |   0.778637 |
+------------------------+------------+
| LogisticRegression     |   0.778426 |
+------------------------+------------+
| LinearSVC              |   0.776427 |
+------------------------+------------+


**9. AJUSTE DE LOS HIPERPARÁMETROS DE LOS MODELOS**

In [40]:
# Crear un EvaluadorClasificaciónMulticlase
evaluator = MulticlassClassificationEvaluator(labelCol="clase", metricName="accuracy")

# Define the classifiers
classifiers = [
    DecisionTreeClassifier(featuresCol="atributos", labelCol='clase'),
    RandomForestClassifier(featuresCol="atributos", labelCol='clase'),
    GBTClassifier(featuresCol="atributos", labelCol='clase'),
    LogisticRegression(featuresCol="atributos", labelCol='clase'),
    LinearSVC(featuresCol="atributos", labelCol='clase')
]

# Crear una lista vacía para almacenar los resultados de la validación cruzada
results = []

# Ajuste de los hiperparámetros de cada clasificador
for classifier in classifiers:
    if isinstance(classifier, DecisionTreeClassifier):
        paramGrid = ParamGridBuilder() \
            .addGrid(classifier.maxDepth, [5, 10, 20]) \
            .build()
    elif isinstance(classifier, RandomForestClassifier):
        paramGrid = ParamGridBuilder() \
            .addGrid(classifier.numTrees, [10, 5, 10]) \
            .addGrid(classifier.maxDepth, [5, 10, 20]) \
            .build()
    elif isinstance(classifier, GBTClassifier):
        paramGrid = ParamGridBuilder() \
            .addGrid(classifier.maxIter, [10, 5, 10]) \
            .addGrid(classifier.maxDepth, [5, 10, 20]) \
            .build()
    elif isinstance(classifier, LogisticRegression):
        paramGrid = ParamGridBuilder() \
            .addGrid(classifier.maxIter, [10, 10, 10]) \
            .addGrid(classifier.regParam, [0.1, 0.01]) \
            .build()
    elif isinstance(classifier, LinearSVC):
        paramGrid = ParamGridBuilder() \
            .addGrid(classifier.maxIter, [10, 10, 10]) \
            .addGrid(classifier.regParam, [0.1, 0.01]) \
            .build()

    # Crear el CrossValidator
    crossval = CrossValidator(estimator=classifier,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=10)

    # Entrenar el modelo con el conjunto de entrenamiento
    cvModel = crossval.fit(train)

    # Almacenar las métricas de evaluación
    metrics = cvModel.avgMetrics
    results.append([classifier.__class__.__name__, metrics])

# Preparar las cabeceras de las tablas
headers = ["Classifier", "Metrics"]

# Imprimir los resultados del ajuste de hiperparámetros en forma de tabla
print(tabulate(results, headers=headers, tablefmt="grid"))

# Utilizamos el conjunto de pruebas para medir el rendimiento del modelo
# utilizando la mejor configuración (bestModel) de los parámetros encontrados
predictions = cvModel.bestModel.transform(test)

# Show the accuracy of the model
print('Accuracy:', evaluator.evaluate(predictions))


+------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Classifier             | Metrics                                                                                                                                                                             |
+========================+=====================================================================================================================================================================================+
| DecisionTreeClassifier | [0.7672221627102239, 0.7502179427134458, 0.6952908613645221]                                                                                                                        |
+------------------------+------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Detener la Session Spark
spark.stop()